In [ ]:
Initialize (two equivalent ways of instantiating our functions)

In [ ]:
self.conv1 = nn.Conv2d(3, 16, 3, padding=1),
self.pool = nn.MaxPool2d(2, 2),
self.relu1 = nn.ReLU()
self.drop1 = nn.Dropout2d(0.2)

In [ ]:
self.conv_block = nn.Sequential(
  nn.Conv2d(3, 16, 3, padding=1),
  nn.MaxPool2d(2, 2),
  nn.ReLU(),
  nn.Dropout2d(0.2)
)

In [ ]:
import torch
import torch.nn as nn

class MyCNN(nn.Module):

  def __init__(self, n_classes):

    super().__init__()

    # Create layers. In this case just a standard MLP
    self.model = nn.Sequential(
      # First conv + maxpool + relu
      nn.Conv2d(3, 16, 3, padding=1),
      nn.MaxPool2d(2, 2),
      nn.ReLU(),
      nn.Dropout2d(0.2),

      # Second conv + maxpool + relu
      nn.Conv2d(16, 32, 3, padding=1),
      nn.MaxPool2d(2, 2),
      nn.ReLU(),
      nn.Dropout2d(0.2),

      # Third conv + maxpool + relu
      nn.Conv2d(32, 64, 3, padding=1),
      nn.MaxPool2d(2, 2),
      nn.ReLU(),
      nn.Dropout2d(0.2),

      # Flatten feature maps
      nn.Flatten(),

      # Fully connected layers. This assumes
      # that the input image was 32x32
      nn.Linear(1024, 128),
      nn.ReLU(),
      nn.Dropout(0.5),
      nn.Linear(128, n_classes)
    )

  def forward(self, x):

    # nn.Sequential will call the layers 
    # in the order they have been inserted
    return self.model(x)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    def __init__(self, n_classes=10):

        super(Net, self).__init__()

        # convolutional layer 1. It sees 3x32x32 image tensor
        # and produces 16 feature maps 32x32 (i.e., a tensor 16x32x32)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        # 2x2 pooling with stride 2. It sees tensors 16x32x32
        # and halves their size, i.e., the output will be 16x16x16
        self.pool1 = nn.MaxPool2d(2, 2)

        # convolutional layer (sees the output of the prev layer, i.e.,
        # 16x16x16 tensor)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)  # -> 32x16x16
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, 2)  # -> 32x8x8

        # convolutional layer
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)  # -> 64x8x8
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2, 2)  # -> 64x4x4

        # linear layer (64 * 4 * 4 -> 500)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 4 * 4, 500)
        self.dp1 = nn.Dropout(0.5)
        self.rl1 = nn.ReLU()

        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(500, n_classes)

    def forward(self, x):

        x = self.relu1(self.pool1(self.conv1(x)))
        x = self.relu2(self.pool2(self.conv2(x)))
        x = self.relu3(self.pool3(self.conv3(x)))

        x = self.flatten(x)

        x = self.rl1(self.dp1(self.fc1(x)))

        x = self.fc2(x)

        return x

In [ ]:
model = Net()

Number of parameters

In [ ]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

Specify loss function and optimizer

In [ ]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
loss = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

Train the network

In [ ]:
from helpers import optimize

In [ ]:
optimize(
    data_loaders,
    model,
    optimizer,
    loss,
    20,
    "cifar10_best_valid.pt",
    interactive_tracking=True
)

Test

In [ ]:
from helpers import one_epoch_test

In [ ]:
test_loss, preds, actuals = one_epoch_test(data_loaders['valid'], model, loss)
from helpers import plot_confusion_matrix

cm = plot_confusion_matrix(preds, actuals, classes)

In [ ]:
print("Accuracy by class:\n")
for i, col in enumerate(cm):
    print(f"    {col:11s}: {cm[col][i] / cm[col].sum():.2f}")

To visualize

In [ ]:
# obtain one batch of test images
dataiter = iter(data_loaders['test'])

for i in range(2):
    images, labels = dataiter.next()
    images.numpy()

    # move model inputs to cuda, if GPU available
    if train_on_gpu:
        images = images.cuda()

    # get sample outputs
    output = model(images)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.numpy()) if not train_on_gpu else np.squeeze(preds_tensor.cpu().numpy())

    # plot the images in the batch, along with predicted and true labels
    fig, subs = plt.subplots(2, 10, figsize=(25, 4))
    for i, ax in enumerate(subs.flatten()):
        imshow(images[i].cpu().numpy(), ax)
        ax.set_title("{} ({})".format(classes[preds[i]], classes[labels[i]]),
                     color=("green" if preds[i]==labels[i].item() else "red"))
        ax.axis("off")